In [2]:
#import required dependencies here:
import pandas as pd
import datetime

#NOTE: before reading in any .csv files, you need to open the .csv file using a simple text editor and remove the first 
#four lines (starting with '#' symbols) in order to get the csv to load correctly

#read in csv here
AQI_2017_df = pd.read_csv("./2019 Source/waqi-covid19-airqualitydata-2017H1.csv")

#check output to see if csv loaded into df correctly
AQI_2017_df.head()

,Date,Country,City,Specie,count,min,max,median,variance
0,2016-12-27,CN,Beijing,co,420,1.0,34.4,7.3,217.26
1,2017-01-28,CN,Beijing,co,434,1.9,49.7,23.5,508.57
2,2017-02-24,CN,Beijing,co,427,1.0,37.1,6.4,374.07
3,2017-03-18,CN,Beijing,co,425,1.0,29.0,11.8,334.03
4,2017-04-16,CN,Beijing,co,407,1.0,17.2,9.1,126.15


In [3]:
#group data set by date, and check the date range of the output
#it looks like for 2018 and earlier years the data might only go to the current date (~7/5) because 
#those dates are comparable to the coronavirus shutdown this year
#2019 should have the full range of dates if all quarters are merged correctly
AQI_2017_df.groupby("Date").count()

,Country,City,Specie,count,min,max,median,variance
Date,,,,,,,,
2016-12-26,2282,2282,2282,2282,2282,2282,2282,2282
2016-12-27,2292,2292,2292,2292,2292,2292,2292,2292
2016-12-28,2310,2310,2310,2310,2310,2310,2310,2310
2016-12-29,2297,2297,2297,2297,2297,2297,2297,2297
2016-12-30,2316,2316,2316,2316,2316,2316,2316,2316
...,...,...,...,...,...,...,...,...
2017-06-28,2219,2219,2219,2219,2219,2219,2219,2219
2017-06-29,2211,2211,2211,2211,2211,2211,2211,2211
2017-06-30,2200,2200,2200,2200,2200,2200,2200,2200


In [4]:
#now that we have checked out the data date ranges, we should narrow the data down to the desired info
#first select by cities that we are interested in
interesting_city_list = ["Los Angeles", "Houston", "Jacksonville", "Manhattan", "Beijing", "Shanghai", "Wuhan", "New Delhi"]
interesting_cities_AQI = AQI_2017_df.loc[AQI_2017_df["City"].isin(interesting_city_list)]

#this should give some idea about how many records there are for each city we are interested in for the year
#the higher the number the more complete the data!
interesting_cities_AQI["City"].value_counts()

Shanghai        1134
Beijing         1129
Wuhan           1116
Los Angeles     1108
Houston          878
Jacksonville     775
Manhattan        742
New Delhi        188
Name: City, dtype: int64

In [5]:
#get subsets of o3 and pm2.5 for each city
interesting_cities_o3_2017 = interesting_cities_AQI.loc[interesting_cities_AQI["Specie"] == "o3"]
interesting_cities_pm25_2017 = interesting_cities_AQI.loc[interesting_cities_AQI["Specie"] == "pm25"]

#can check to see if we are missing data for any days for either o3 or pm2.5
print(interesting_cities_o3_2017["City"].value_counts())
print(interesting_cities_pm25_2017["City"].value_counts())

#needed to beef up the AQI conversion table past 500 so that higher AQIs could be converted to concentrations
#used linear regression on the last 50 AQI values (450-500) to extrapolate AQI to concentration conversions for AQI values past 
#500, up to 1000 - but please note that this is an assumption!
print(interesting_cities_pm25_2017["max"].max())

Shanghai        189
Houston         189
Los Angeles     189
Wuhan           186
Jacksonville    186
Beijing         184
Manhattan       184
Name: City, dtype: int64
Shanghai        189
Houston         189
Beijing         189
Los Angeles     189
Manhattan       188
New Delhi       188
Jacksonville    187
Wuhan           186
Name: City, dtype: int64
999.0


In [9]:

pm25_conversion_df = pd.read_csv("./Conversion CSVs/pm25_conversion.csv")
o3_conversion_df = pd.read_csv("./Conversion CSVs/o3_conversion.csv")

In [10]:
#conversion from aqi to concentration for pm2.5
#variance omitted because that calculation is not directly proportional to AQI and not that relevant for our project

#function AQI_to_Conc takes an o3 or an pm25 dataframe as an argument
#returns a new dataframe with the min, max, and median values converted to measures with units
#also drops the variance column for the new dataframe returned
def AQI_to_Conc(df):
    #setup dictionary to choose how to convert input data
    types = {"o3":o3_conversion_df, "pm25":pm25_conversion_df}
    
    #picks conversion df dynamically
    pollutant = df["Specie"].iloc[0]
    conversion_df = types[pollutant]
    
    #copy input dataframe -> will be changed by function
    updated_df = df.copy()
    
    #get the units for the concentration measurements
    units = conversion_df.columns[0].replace(pollutant, "")
    
    #add the new columns
    update_columns = ["min", "max", "median"]
    for col in update_columns:
        #note that we need to round the AQI to get an integer for row lookup
        updated_df[col + units] = updated_df[col].apply(lambda x: conversion_df[pollutant + units][round(x)])
    
    #drop the old columns (and "variance")
    updated_df = updated_df.drop(axis=1, columns=(update_columns+["variance"]))

    return updated_df
    
#get new dataframes for the concentrations of the pollutants using the above function
#only takes the dataframe for one pollutant as an argument
interesting_cities_pm25_2017_conc = AQI_to_Conc(interesting_cities_pm25_2017)
interesting_cities_o3_2017_conc = AQI_to_Conc(interesting_cities_o3_2017)

print(interesting_cities_pm25_2017_conc.head())
interesting_cities_o3_2017_conc.head()


           Date Country     City Specie  count  min (ug/m3)  max (ug/m3)  \
751  2017-02-02      CN  Beijing   pm25    436          3.1        414.0   
752  2017-02-13      CN  Beijing   pm25    452          5.0        285.8   
753  2017-02-27      CN  Beijing   pm25    444          3.1        190.8   
754  2017-03-11      CN  Beijing   pm25    276          5.0        177.7   
755  2017-03-26      CN  Beijing   pm25    414          3.1         29.2   

     median (ug/m3)  
751            88.4  
752            90.3  
753            67.1  
754            35.9  
755            13.0  


,Date,Country,City,Specie,count,min (ppb),max (ppb),median (ppb)
378,2017-01-28,CN,Beijing,o3,395,1.0,55.0,3.0
379,2017-03-25,CN,Beijing,o3,336,1.0,54.0,25.0
380,2017-03-29,CN,Beijing,o3,296,1.0,64.0,29.0
381,2017-03-31,CN,Beijing,o3,438,1.0,51.0,28.0
382,2017-04-08,CN,Beijing,o3,349,1.0,41.0,12.0


In [11]:
#output the pm25 concentration data for our cities for the year 2018
interesting_cities_pm25_2017_conc.to_csv("./2019 Cleaned/our_cities_pm25_2017.csv", index=False)

#output the o3 concentration data for our cities for the year 2018
interesting_cities_o3_2017_conc.to_csv("./our_cities_o3_2017.csv", index=False)

In [ ]:
#select date range to measure PM2.5 for full shutdown - include all years
#messy right now - turn into a function!

#use Wuhan as an example
earliest_year = min(all_pm25_df.index.year)
latest_year = max(all_pm25_df.index.year)
shutdown_start_date = (1, 23)#"1/23"
shutdown_end_date = (4, 8)#"4/8"
mask = ((pd.Series(map(lambda x: x.month <= shutdown_start_date[0], all_pm25_df.index.date), index=all_pm25_df.index)) &
                                            ((pd.Series(map(lambda x: x.day < shutdown_start_date[1], all_pm25_df.index.date), index=all_pm25_df.index))))
#first get dates after the start date for all years
shutdown_time_period_pm_df = all_pm25_df.loc[~mask, :]

#remove the later months
shutdown_time_period_pm_df = shutdown_time_period_pm_df.loc[shutdown_time_period_pm_df.index.month<=shutdown_end_date[0]]

#now get dates before the end date
mask2 = ((pd.Series(map(lambda x: x.month == shutdown_end_date[0], shutdown_time_period_pm_df.index.date), index=shutdown_time_period_pm_df.index)) &
                                            ((pd.Series(map(lambda x: x.day >= shutdown_end_date[1], shutdown_time_period_pm_df.index.date), index=shutdown_time_period_pm_df.index))))

shutdown_time_period_pm_df = shutdown_time_period_pm_df.loc[~mask2, :]

shutdown_time_period_pm_df = shutdown_time_period_pm_df.loc[shutdown_time_period_pm_df["City"] == "Wuhan"]
print(shutdown_time_period_pm_df.head())
shutdown_time_period_pm_df.tail(10)